In [6]:
### Initialization & Loading:

In [5]:
# Imports should ideally be grouped at the top
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, explode # Added required functions here
import seaborn as sns
import matplotlib.pyplot as plt # Keep even if not directly used, seaborn uses it
import pandas as pd
# import json # Not used in the provided snippet

spark = SparkSession.builder.appName("TVShowAnalysis").getOrCreate()

# Define file_path BEFORE using it
file_path = "/FileStore/tables/tv_shows.json"

# Use spark.read, not sqlContext.read (modern API)
# Remove the first attempt to read before file_path was defined
data = spark.read.json(file_path)

# Look at a data row (optional, good for check)
# data.show(1, truncate=False, vertical=True) # Alternative way to view a row nicely
print(data.take(1))

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
### Row Counts & Duplicates:

In [ ]:
# Return number of rows
initial_count = data.count()
print(f"Initial row count: {initial_count}")

# Remove duplicate rows, and return row count
# dropDuplicates() returns a NEW DataFrame, it doesn't modify 'data' in-place
data_deduplicated = data.dropDuplicates()
deduplicated_count = data_deduplicated.count()
print(f"Row count after dropDuplicates: {deduplicated_count}")

# If you intend to use the deduplicated data from now on:
# data = data_deduplicated
# Otherwise, subsequent operations use the original 'data'

In [ ]:
### Schema Inspection:

In [ ]:
# View column datatypes (returns list of tuples)
print(data.dtypes)

# View schema info (prints tree structure)
# printSchema is a method, needs parentheses ()
data.printSchema()

In [ ]:
### Exploring Names & Languages:

In [ ]:
# View a selection of title names
display(data.select("name").limit(10))

# Display total count of rows (or non-null names)
# data.select("name").count() is the same as data.count() unless names can be null
# Maybe you wanted distinct names? data.select("name").distinct().count()
print(f"Total shows (data.count()): {data.count()}")

# Count number of shows in each language
# Filter out rows where 'language' column is not null
filtered_data = data.filter(col('language').isNotNull())

# Count distinct languages
distinct_language_count = filtered_data.select('language').distinct().count()
print(f"Total distinct languages found: {distinct_language_count}") # Matches the 73 mentioned in summary

# Group by 'language' column and count occurrences, get top 7
# You already filtered nulls, so you can apply groupBy directly to filtered_data
language_counts = filtered_data.groupBy('language').count().orderBy(desc('count'))

# Display the top 7
display(language_counts.limit(7))

In [ ]:
### Genre Analysis:

In [ ]:
# Explode genres directly from the main DataFrame
# Assuming 'genres' is an array column in the 'data' DataFrame
# No need for the temporary view and SQL unless preferred

# Check if 'genres' column exists and is of array type first (optional but good practice)
if 'genres' in dict(data.dtypes) and data.schema['genres'].dataType.typeName() == 'array':
    distinct_genres = data.select(explode("genres").alias("genre")).distinct()
    print("Distinct Genres:")
    display(distinct_genres) # Use display for better formatting in Databricks
else:
    print("Column 'genres' not found or is not an ArrayType.")

# --- Alternative using SQL (if preferred) ---
# data.createOrReplaceTempView("tv_shows_view")
# distinct_genres_sql = spark.sql("""
#     SELECT DISTINCT exploded_genre
#     FROM tv_shows_view
#     LATERAL VIEW explode(genres) exploded_table AS exploded_genre
# """)
# print("Distinct Genres (SQL method):")
# display(distinct_genres_sql)

In [ ]:
### Show Type Analysis & Visualization:

In [ ]:
# View count by show-type
type_data = data.groupby('type').count().orderBy(desc('count')) # Added order for consistency
print("Counts by Show Type:")
display(type_data)

# Convert to Pandas for plotting (fine for small aggregated data)
types_pandas = type_data.toPandas()

# Plotting
sns.set_theme(style="whitegrid") # Use set_theme for modern seaborn
plt.figure(figsize=(8, 4)) # Optional: Adjust figure size
sns.barplot(data=types_pandas, x='count', y='type')
plt.title('Number of Shows by Type') # Add a title
plt.show() # Explicitly show plot

In [ ]:
### Runtime Analysis & Visualization:

In [ ]:
# Analyze Average Runtime distribution
# Grouping by runtime and counting is valid but doesn't directly give the distribution plot desired
# x_data = data.groupby('averageRuntime').count()
# display(x_data.limit(5))

# For plotting the distribution of runtimes, work with the column directly
# Filter out potential nulls or invalid values if necessary
runtime_data_pd = data.select("averageRuntime").filter(col("averageRuntime").isNotNull()).toPandas()

# Plot the distribution (KDE)
sns.displot(data=runtime_data_pd, x='averageRuntime', kind="kde")
plt.title('Distribution of Average Show Runtime (minutes)')
plt.show()

# -- If the dataset is HUGE, sample before toPandas --
# sample_fraction = 0.1 # Adjust as needed
# runtime_data_pd_sampled = data.select("averageRuntime") \
#                                .filter(col("averageRuntime").isNotNull()) \
#                                .sample(withReplacement=False, fraction=sample_fraction) \
#                                .toPandas()
# sns.displot(data=runtime_data_pd_sampled, x='averageRuntime', kind="kde")
# plt.title(f'Distribution of Average Show Runtime (Sampled {int(sample_fraction*100)}%)')
# plt.show()

In [ ]:
## 